# Importações

In [0]:
from pyspark.sql import functions as F

# Carregar os dados da camada Silver

In [0]:
df_silver = spark.table("silver_ooni")

# Dimensões (Esquema Estrela)

In [0]:
# Dimensão País
dim_country = (
    df_silver
    .select("probe_cc")
    .where(F.col("probe_cc").isNotNull())
    .dropDuplicates()
    .withColumnRenamed("probe_cc", "country_code")
)

dim_country.write.mode("overwrite").saveAsTable("gold_dim_country")

# Dimensão teste
dim_test = (
    df_silver
    .select("test_name")
    .where(F.col("test_name").isNotNull())
    .dropDuplicates()
    .withColumnRenamed("test_name", "test")
)

dim_test.write.mode("overwrite").saveAsTable("gold_dim_test")

# Dimensão tempo
dim_time = (
    df_silver
    .select("measurement_ts", "year", "month", "day")
    .withColumn("date", F.to_date("measurement_ts"))
    .dropDuplicates()
)

dim_time.write.mode("overwrite").saveAsTable("gold_dim_time")

# Dimensão Provedor 
dim_asn = (
    df_silver
    .select("probe_asn")
    .where(F.col("probe_asn").isNotNull())
    .dropDuplicates()
    .withColumnRenamed("probe_asn", "asn")
)

dim_asn.write.mode("overwrite").saveAsTable("gold_dim_asn")

# Criar Tabela Fato

In [0]:
# Fato de Medições

dbutils.fs.rm("/user/hive/warehouse/gold_fact_measurement", recurse=True)

fact_measurement = (
    df_silver
    .withColumn("date", F.to_date("measurement_ts"))
    .select(
        F.col("measurement_uid").cast("string"),
        F.col("probe_cc").cast("string").alias("country_code"),
        F.col("test_name").cast("string"),
        F.col("probe_asn").cast("int").alias("asn"),
        F.col("date").cast("date"),

        F.col("is_anomalous").cast("boolean"),
        F.col("is_confirmed_blocking").cast("boolean"),
        F.col("has_failure").cast("boolean"),

        F.col("blocking_global").cast("double"),
        F.col("blocking_isp").cast("double"),
        F.col("blocking_local").cast("double"),
        F.col("blocking_general").cast("double"),
        F.col("blocking_type").cast("string")
    )
)

(
    fact_measurement.write
        .format("delta")
        .mode("overwrite")
        .saveAsTable("gold_fact_measurement")
)

# Métricas Analíticas

In [0]:
%sql
-- Comparação regional 

DROP TABLE IF EXISTS gold_censorship_by_country_test;

CREATE TABLE gold_censorship_by_country_test AS
SELECT
  country_code,
  test_name,
  COUNT(*)                                                   AS total_measurements,
  SUM(CAST(is_anomalous AS INT))                             AS anomaly_count,
  ROUND(
    SUM(CAST(is_anomalous AS INT)) / COUNT(*),
    4
  )                                                          AS anomaly_rate
FROM gold_fact_measurement
WHERE date >= add_months(current_date(), -12)
GROUP BY country_code, test_name;



In [0]:
%sql
select * 
from gold_censorship_by_country_test

In [0]:
%sql
-- Brasil x Média regional

DROP TABLE IF EXISTS gold_brazil_vs_region;

CREATE OR REPLACE TABLE gold_brazil_vs_region AS
SELECT
  test_name,
  AVG(anomaly_rate)                                  AS avg_region_rate,
  MAX(CASE WHEN country_code = 'BR' THEN anomaly_rate END) AS brazil_rate
FROM gold_censorship_by_country_test
GROUP BY test_name;


In [0]:
%sql
select *
from gold_brazil_vs_region

In [0]:
%sql
-- Impacto temporal (volume de bloqueios)

DROP TABLE IF EXISTS gold_blocking_over_time;

CREATE TABLE gold_blocking_over_time AS
SELECT
  date,
  country_code,
  COUNT(*)                                   AS total_measurements,
  SUM(CAST(is_confirmed_blocking AS INT))    AS confirmed_blocks
FROM gold_fact_measurement
GROUP BY date, country_code;

In [0]:
%sql
select * from gold_blocking_over_time

In [0]:
%sql
--Análise de provedores (Brasil)

DROP TABLE IF EXISTS gold_blocking_by_asn_br;

CREATE TABLE gold_blocking_by_asn_br AS
SELECT
  asn,
  COUNT(*)                                    AS total_measurements,
  SUM(CAST(is_anomalous AS INT))              AS anomaly_count,
  ROUND(
    SUM(CAST(is_anomalous AS INT)) / COUNT(*),
    4
  )                                           AS anomaly_rate
FROM gold_fact_measurement
WHERE country_code = 'BR'
GROUP BY asn
ORDER BY anomaly_rate DESC;



In [0]:
%sql
select * from gold_blocking_by_asn_br

In [0]:
%sql
-- qualidade dos dados

DROP TABLE IF EXISTS gold_data_quality_summary;

CREATE TABLE gold_data_quality_summary AS
SELECT
  COUNT(*) AS total_records,

  ROUND(
    SUM(CASE WHEN country_code IS NULL THEN 1 ELSE 0 END) / COUNT(*),
    4
  ) AS pct_country_missing,

  ROUND(
    SUM(CASE WHEN asn IS NULL THEN 1 ELSE 0 END) / COUNT(*),
    4
  ) AS pct_asn_missing
FROM gold_fact_measurement;



In [0]:
%sql
select * from gold_data_quality_summary